In [1]:
!pip install mlflow pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.6 MB/s eta 0:00:00


## Downloading the Tiny ImageNet Dataset

In [2]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

--2025-10-28 06:39:40--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2025-10-28 06:39:41--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  3.82MB/s    in 57s     

2025-10-28 06:40:38 (4.18 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



## Importing the necessary packages

In [3]:
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from PIL import Image

## Setting up MLflow with ngrok

In [4]:
from google.colab import userdata
ngrok_token = userdata.get("ngrokToken")

In [5]:
os.environ["MLFLOW_ENABLE_HOST_CHECKING"] = "false"

In [15]:
# !pkill -f ngrok

In [16]:
get_ipython().system_raw("mlflow ui --port 2000 &")
mlflow.set_tracking_uri("http://localhost:2000")
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_token)

In [17]:
public_url = ngrok.connect(2000).public_url
print(public_url)

https://dino-advisory-unprimitively.ngrok-free.dev


## Configuring PyTorch to use the GPU if available

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## Loading the datset

In [10]:
data_dir = "/content/tiny-imagenet-200"
num_classes = 200
batch_size = 64

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform_train)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

## Building the model

In [19]:
model = models.efficientnet_b0(weights='IMAGENET1K_V1')

model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Training the model

In [12]:
mlflow.set_experiment("EfficientNet_TinyImageNet")

with mlflow.start_run(run_name="EfficientNet-B0-TinyImageNet"):

    mlflow.log_param("model", "efficientnet_b0")
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_classes", num_classes)

    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct, total = 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        avg_train_loss = train_loss / len(train_loader)

        # Log metrics
        mlflow.log_metric("train_loss", avg_train_loss, step=epoch)
        mlflow.log_metric("train_accuracy", train_acc, step=epoch)

        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_train_loss:.4f} | Acc: {train_acc:.2f}%")

    # Save model with MLflow
    mlflow.pytorch.log_model(model, "model")


2025/10/28 04:26:14 INFO mlflow.tracking.fluent: Experiment with name 'EfficientNet_TinyImageNet' does not exist. Creating a new experiment.
100%|██████████| 1563/1563 [11:35<00:00,  2.25it/s]


Epoch [1/5] Loss: 2.8617 | Acc: 35.30%


100%|██████████| 1563/1563 [11:28<00:00,  2.27it/s]


Epoch [2/5] Loss: 2.2814 | Acc: 46.00%


100%|██████████| 1563/1563 [11:32<00:00,  2.26it/s]


...

Epoch [4/5] Loss: 1.9654 | Acc: 52.67%


100%|██████████| 1563/1563 [11:30<00:00,  2.26it/s]
2025/10/28 05:23:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch [5/5] Loss: 1.8875 | Acc: 54.28%


2025/10/28 05:23:55 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.8.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/28 05:24:04 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torchvision==0.23.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/28 05:24:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run EfficientNet-B0-TinyImageNet at: http://localhost:2000/#/experiments/224451195676655221/runs/0efbeb13115647e79557294347de6a71
🧪 View experiment at: http://localhost:2000/#/experiments/224451195676655221


In [14]:
model.eval()
val_correct, val_total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_correct += (preds == labels).sum().item()
        val_total += labels.size(0)
val_acc = 100 * val_correct / val_total

mlflow.log_metric("val_accuracy", val_acc)

## Saving the model

In [13]:
torch.save(model.state_dict(), "efficientnet_b0_tinyimagenet.pth")
print("Model saved successfully!")

Model saved successfully!


## Loading the model and testing on custom data

In [11]:
# Mapping from wnid to class name
label_dict = {}
with open("/content/tiny-imagenet-200/words.txt", "r") as f:
    for line in f:
        wnid, name = line.strip().split("\t")
        label_dict[wnid] = name

# Loading the trained model
model = models.efficientnet_b0(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.load_state_dict(torch.load("efficientnet_b0_tinyimagenet.pth", map_location=device))
model.to(device)
model.eval()

# Preprocessing image
transform_infer = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

img_path = "/content/example2.jpg"
image = Image.open(img_path).convert("RGB")
image_t = transform_infer(image).unsqueeze(0).to(device)

# Predicting on custom data
with torch.no_grad():
    outputs = model(image_t)
    probs = torch.nn.functional.softmax(outputs, dim=1)
    pred_class = probs.argmax(dim=1).item()

# Getting class name from wnid
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}
wnid = idx_to_class[pred_class]
class_name = label_dict.get(wnid, "Unknown class")

print(f"Predicted class: {class_name} ({wnid})")


Predicted class: scorpion (n01770393)
